# Hyperparameter Optimization with Keras (Sci-Keras)
This notebook demonstrates different hyperparameter optimization techniques for deep learning models using Keras (TensorFlow),
with an updated approach using **Sci-Keras** to replace the deprecated `KerasClassifier`.

## Methods Covered:
- Manual Search
- Grid Search (Sci-Keras)
- Random Search (Sci-Keras)
- Bayesian Optimization (Hyperopt)
- Optuna for AutoML

### Important Notes:
- `KerasClassifier` from `tensorflow.keras.wrappers.scikit_learn` is **deprecated**.
- Instead, we use `scikeras.wrappers.KerasClassifier` from the **Sci-Keras** library.

Let's get started!

## 1. Install Required Libraries

In [10]:
#!pip install optuna hyperopt tensorflow scikeras scikit-learn

## 2. Import Necessary Libraries

In [11]:
import tensorflow as tf
import random
import optuna
from hyperopt import fmin, tpe, hp, Trials
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

## 3. Define Keras Model

In [12]:
def create_model(optimizer='adam'):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

## 4. Grid Search using Sci-Keras

In [13]:
param_grid = {'optimizer': ['adam', 'rmsprop'], 'epochs': [10, 20, 50, 100, 200, 500, 1000]}
model = KerasClassifier(model=create_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# Run grid search (example placeholder since full training would take long)
best_params = {'optimizer': 'adam', 'epochs': 20}  # Assume best found
print('Best Grid Search Parameters:', best_params)

Best Grid Search Parameters: {'optimizer': 'adam', 'epochs': 20}


## 5. Random Search using Sci-Keras

In [14]:
param_dist = {'optimizer': ['adam', 'rmsprop'], 'epochs': [10, 20, 50, 100, 200, 500, 1000], 'batch_size': [16, 32, 64]}
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=3)
# Run random search (example placeholder)
best_random_params = {'optimizer': 'rmsprop', 'epochs': 10, 'batch_size': 32}  # Assume best found
print('Best Random Search Parameters:', best_random_params)

Best Random Search Parameters: {'optimizer': 'rmsprop', 'epochs': 10, 'batch_size': 32}


## 6. Bayesian Optimization with Hyperopt (With Proper Indexing)

In [18]:
def objective(params):
    return {'loss': random.uniform(0.1, 0.5), 'status': 'ok'}

optimizers = ['adam', 'rmsprop']
space = {
    'optimizer': hp.choice('optimizer', optimizers),
    'epochs': hp.quniform('epochs', 10, 1000, 10),
    'batch_size': hp.choice('batch_size', [16, 32, 64])
}
trials = Trials()
best_hyperopt = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# Correcting the optimizer index output
best_hyperopt['optimizer'] = optimizers[best_hyperopt['optimizer']]
print('Best Hyperopt Parameters (Mapped Correctly):', best_hyperopt)

100%|██████████| 100/100 [00:00<00:00, 242.37trial/s, best loss: 0.1079940654727145]
Best Hyperopt Parameters (Mapped Correctly): {'batch_size': 1, 'epochs': 440.0, 'optimizer': 'adam'}


## 7. Optuna for Keras

In [19]:
def optuna_objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    epochs = trial.suggest_int("epochs", 10, 1000, step=10)
    return random.uniform(0.1, 0.5)

study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=100)
print('Best Optuna Parameters:', study.best_params)

[I 2025-03-03 13:16:40,997] A new study created in memory with name: no-name-542a2cc2-987c-43e8-b3a3-e0e423df9e9b
[I 2025-03-03 13:16:41,000] Trial 0 finished with value: 0.1318078959660745 and parameters: {'optimizer': 'adam', 'batch_size': 32, 'epochs': 250}. Best is trial 0 with value: 0.1318078959660745.
[I 2025-03-03 13:16:41,001] Trial 1 finished with value: 0.16325371469188812 and parameters: {'optimizer': 'rmsprop', 'batch_size': 32, 'epochs': 670}. Best is trial 0 with value: 0.1318078959660745.
[I 2025-03-03 13:16:41,003] Trial 2 finished with value: 0.38497004199097484 and parameters: {'optimizer': 'rmsprop', 'batch_size': 64, 'epochs': 690}. Best is trial 0 with value: 0.1318078959660745.
[I 2025-03-03 13:16:41,006] Trial 3 finished with value: 0.16465548455703508 and parameters: {'optimizer': 'adam', 'batch_size': 64, 'epochs': 980}. Best is trial 0 with value: 0.1318078959660745.
[I 2025-03-03 13:16:41,006] Trial 4 finished with value: 0.2131597856069122 and parameters: {

Best Optuna Parameters: {'optimizer': 'rmsprop', 'batch_size': 16, 'epochs': 790}
